In [9]:
import requests
import datetime

import pandas as pd
import zipfile

import mlflow
import os

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [10]:
#set up tracking server
TRACKING_SERVER_HOST = "34.171.118.161" #external IP reserved in GCP
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [11]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://34.171.118.161:5000'


In [12]:
data_file = "../data/processed/202304-usage.parquet"

In [13]:
mlflow.set_experiment("experiment-1")

<Experiment: artifact_location='gs://mlops-divvy-experiment-tracking/mlruns/1', creation_time=1721225054205, experiment_id='1', last_update_time=1721225054205, lifecycle_stage='active', name='experiment-1', tags={}>

In [23]:
with mlflow.start_run():
    #load prepared data
    df = pd.read_parquet(data_file)
    mlflow.log_param("data_file", data_file)

    features_num = df[['hour', 'day_of_week']]
    features_cat = df[['station_name', 'day_of_week']]

    target = df['net_usage']

    #features = pd.get_dummies(features, columns=['station_name', 'day_of_week']) #encoding categorical
    # encoder = OneHotEncoder(handle_unknown='ignore')
    # encoded_cat = encoder.fit_transform(features_cat[['station_name','day_of_week']])

    # encoded_df = pd.DataFrame(encoded_cat)

    # features_encoded = pd.concat([features_num, encoded_df], axis=1)
    # print(features_encoded)

    # not working - needs to update - jul 18 2024

    split_params = {"test_size": 0.2, "random_state": 42}
    X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, **split_params)

    params = {"n_estimators": 10, "random_state": 42}
    mlflow.log_params(params)

    rf = RandomForestRegressor(**params).fit(X_train, y_train)
    
    y_pred = rf.predict(X_test)
    mlflow.log_metric("mse", mean_squared_error(y_test, y_pred))

    mlflow.sklearn.log_model(rf, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

mlflow.search_experiments()

       hour day_of_week                                   0
0       0.0      Sunday       (0, 0)\t1.0\n  (0, 1058)\t1.0
1       0.0    Thursday       (0, 0)\t1.0\n  (0, 1059)\t1.0
2       0.0     Tuesday       (0, 0)\t1.0\n  (0, 1060)\t1.0
3       0.0   Wednesday       (0, 0)\t1.0\n  (0, 1061)\t1.0
4       1.0      Sunday       (0, 0)\t1.0\n  (0, 1058)\t1.0
...     ...         ...                                 ...
70045  17.0    Saturday    (0, 1054)\t1.0\n  (0, 1057)\t1.0
70046  18.0     Tuesday    (0, 1054)\t1.0\n  (0, 1060)\t1.0
70047  19.0     Tuesday    (0, 1054)\t1.0\n  (0, 1060)\t1.0
70048  21.0      Sunday    (0, 1054)\t1.0\n  (0, 1058)\t1.0
70049  22.0    Saturday    (0, 1054)\t1.0\n  (0, 1057)\t1.0

[70050 rows x 3 columns]


TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.